# Twitter API Premium v1.1: Sandbox

In [12]:
# Import dependencies
import tweepy
import pandas as pd
import json

In [3]:
# Import API client token
import os
from dotenv import load_dotenv
load_dotenv()

twit_token = os.getenv('twit_token')

In [95]:
auth = tweepy.OAuth2BearerHandler(twit_token)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [207]:
# Code reproduced from Twitter developer API documentation [4]
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type

parsedTweets = []
rawTweets = []

def parse_tweets(status):
    for tweet in status:
        
        rawTweets.append(tweet)
        
        if determine_tweet_type(tweet._json) == 'Retweet':
            if 'extended_tweet' in tweet._json['retweeted_status']:
                full_text = tweet._json['retweeted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['retweeted_status']['text']
               

        elif determine_tweet_type(tweet._json) == 'Quote Tweet':
            if 'extended_tweet' in tweet._json['quoted_status']:
                full_text = tweet._json['quoted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['quoted_status']['text']
                
        else:
            if 'extended_tweet' in tweet._json:
                full_text = tweet._json['extended_tweet']['full_text']
            else:
                full_text = tweet._json['text']

        mydict = { "tweet_id": tweet._json["id_str"],
                       "date":tweet._json["created_at"],
                       "full_text": full_text,
                       "tweet_type": determine_tweet_type(tweet._json),
                        "reply_count": tweet._json["reply_count"], #Number of times Tweet has been replied to
                       "quote_count": tweet._json["quote_count"], # Number of times Tweet has been quoted
                       "likes_count": tweet._json["favorite_count"], #Number of times Tweet has been liked 
                       "retweet_counts": tweet._json["retweet_count"], #Number of times this Tweet has been retweeted
                       "hyperlink": "https://twitter.com/twitter/status/" + tweet._json["id_str"]
              }
        
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list

In [402]:
# Twitter API v2 Authentication
client = tweepy.Client(bearer_token=twit_token)

query = '(gun OR guns)  (#guncontrol OR #gunrights OR #gunsense OR #2a OR #gunviolence OR #nra OR #guns OR #shooting OR #firearms OR #gunsafety OR #uvalde \
OR "gun control" OR "2nd amendment" OR "gun violence" OR "right to bear" OR "nra" OR "gun safety" OR "gun rights" OR "good guy with a gun" \
OR "assault weapons" OR "ban guns" OR "ban assault" OR "school shootings" OR "ar-15" OR "mass Shootings") lang:en -is:retweet -is:quote'
counts = client.get_recent_tweets_count(query=query, granularity='day')

# OR guns OR gun

for i in counts.data:
    print(f"{i}")

{'end': '2022-11-04T00:00:00.000Z', 'start': '2022-11-03T04:56:00.000Z', 'tweet_count': 45821}
{'end': '2022-11-05T00:00:00.000Z', 'start': '2022-11-04T00:00:00.000Z', 'tweet_count': 67369}
{'end': '2022-11-06T00:00:00.000Z', 'start': '2022-11-05T00:00:00.000Z', 'tweet_count': 57016}
{'end': '2022-11-07T00:00:00.000Z', 'start': '2022-11-06T00:00:00.000Z', 'tweet_count': 41258}
{'end': '2022-11-08T00:00:00.000Z', 'start': '2022-11-07T00:00:00.000Z', 'tweet_count': 63449}
{'end': '2022-11-09T00:00:00.000Z', 'start': '2022-11-08T00:00:00.000Z', 'tweet_count': 72647}
{'end': '2022-11-10T00:00:00.000Z', 'start': '2022-11-09T00:00:00.000Z', 'tweet_count': 76478}
{'end': '2022-11-10T04:56:00.000Z', 'start': '2022-11-10T00:00:00.000Z', 'tweet_count': 26108}


# --------------------------------------------------------------------------------------------------------------------------------------------------------

In [208]:
query = '(#guncontrol OR #gunrights OR #gunsense OR #2a OR #gunviolence) lang:en' #paid -is:retweet -is:reply
#response = api.search_30_day(label='tweetsentiment', query=query, fromDate=202211030000, maxResults=10) #  toDate=202211040000,
# response = []
# for status in tweepy.Cursor(api.search_30_day(label='tweetsentiment', query=query, toDate='202211040000', maxResults=100),
#                             count=100).pages(2):
#     response.append(status.id)
    

parsedTweets = []
rawTweets = []

# for page in tweepy.Cursor(api.search_30_day, label='tweetsentiment', query=query).pages(2):
    
#     parse_tweets(page)

status = tweepy.Cursor(api.search_30_day, label='tweetsentiment', query=query, fromDate='202211030000', toDate='202211040000').items(100)

parse_tweets(status)

    
tweet_df = pd.DataFrame(parsedTweets)

In [209]:
tweet_df['full_text'][99]

'𝐅𝐢𝐫𝐬𝐭 𝐍𝐚𝐭𝐢𝐨𝐧𝐬 𝐂𝐡𝐢𝐞𝐟 𝐓𝐞𝐫𝐫𝐲 𝐓𝐞𝐞𝐠𝐞𝐞 𝐞𝐱𝐩𝐫𝐞𝐬𝐬𝐞𝐬 𝐜𝐨𝐧𝐜𝐞𝐫𝐧𝐬 𝐰𝐢𝐭𝐡 𝐂-𝟐𝟏\nRegional Chief @Terry_Teegee of @BCAFN had concerns #C21 could restrict access to guns historically used by First Nations #cdnpoli #guncontrol\nhttps://t.co/vmSOcIUWSI'

In [212]:
rawTweets_saved = rawTweets
len(rawTweets_saved)

100

In [214]:
parsedTweets_saved = parsedTweets
len(parsedTweets_saved)

100

In [217]:
len(parsedTweets)

100

In [218]:
len(rawTweets)

100

In [352]:
print(json.dumps(rawTweets[33]._json, indent=2))

{
  "created_at": "Thu Nov 03 23:31:02 +0000 2022",
  "id": 1588312826862452737,
  "id_str": "1588312826862452737",
  "text": "California #GunControl fostered this https://t.co/Ql9sMbKeqX",
  "display_text_range": [
    0,
    36
  ],
  "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
  "truncated": false,
  "in_reply_to_status_id": null,
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id": null,
  "in_reply_to_user_id_str": null,
  "in_reply_to_screen_name": null,
  "user": {
    "id": 1519006246191304705,
    "id_str": "1519006246191304705",
    "name": "Triggerology",
    "screen_name": "Triggerology_FT",
    "location": "Southern California",
    "url": "https://linktr.ee/triggerology",
    "description": "2A Instructor & Advocate, \ud83d\ude0eMinistry of FREEDOM",
    "translator_type": "none",
    "protected": false,
    "verified": false,
    "followers_count": 126,
    "friends_count": 134,
    "listed_count": 1,
    "

In [249]:
 rawTweets[96]._json["user"]["location"]

'California, USA'

In [266]:
range(len(rawTweets[96]._json["entities"]["hashtags"]))

range(0, 3)

In [264]:
parsedTweets[96]['full_text']

'15-month-old boy’s death now ruled a homicide in Rocky Mount, police say #GunControlNow #2a #nc \n\nhttps://t.co/Jt67E2LjdF https://t.co/Y8BUtBpYRJ via @GoogleNews'

In [280]:
rawTweets[96]._json["extended_tweet"]["entities"]["user_mentions"][0]["screen_name"]

'googlenews'

In [281]:
mentions

['@googlenews']

In [284]:
mentions = []
for mention in range(len(rawTweets[96]._json["extended_tweet"]["entities"]["user_mentions"])):
    mentions.append("@" + rawTweets[96]._json["extended_tweet"]["entities"]["user_mentions"][mention]["screen_name"])
mentions

['@googlenews']

In [270]:
hashtags = []
for hashtag in range(len(rawTweets[96]._json["extended_tweet"]["entities"]["hashtags"])):
    hashtags.append("#" + rawTweets[96]._json["extended_tweet"]["entities"]["hashtags"][hashtag]['text'])
hashtags

['#GunControlNow', '#2a', '#nc']

In [207]:
# Code reproduced from Twitter developer API documentation [4]
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type

parsedTweets = []
rawTweets = []

def parse_tweets(status):
    for tweet in status:
        
        rawTweets.append(tweet)
        
        if determine_tweet_type(tweet._json) == 'Retweet':
            if 'extended_tweet' in tweet._json['retweeted_status']:
                full_text = tweet._json['retweeted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['retweeted_status']['text']
               

        elif determine_tweet_type(tweet._json) == 'Quote Tweet':
            if 'extended_tweet' in tweet._json['quoted_status']:
                full_text = tweet._json['quoted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['quoted_status']['text']
                
        else:
            if 'extended_tweet' in tweet._json:
                full_text = tweet._json['extended_tweet']['full_text']
            else:
                full_text = tweet._json['text']

        mydict = { "tweet_id": tweet._json["id_str"],
                       "date":tweet._json["created_at"],
                       "full_text": full_text,
                       "tweet_type": determine_tweet_type(tweet._json),
                        "reply_count": tweet._json["reply_count"], #Number of times Tweet has been replied to
                       "quote_count": tweet._json["quote_count"], # Number of times Tweet has been quoted
                       "likes_count": tweet._json["favorite_count"], #Number of times Tweet has been liked 
                       "retweet_counts": tweet._json["retweet_count"], #Number of times this Tweet has been retweeted
                       "hyperlink": "https://twitter.com/twitter/status/" + tweet._json["id_str"]
              }
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list

In [216]:
parsedTweets = []
rawTweets = []

parse_tweets(rawTweets_saved)
tweet_df2 = pd.DataFrame(parsedTweets)
tweet_df2

,tweet_id,date,full_text,tweet_type,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1588320111932252160,Thu Nov 03 23:59:58 +0000 2022,"Just before the midterm election, the White Ho...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15883201119...
1,1588319976142041088,Thu Nov 03 23:59:26 +0000 2022,"Just before the midterm election, the White Ho...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15883199761...
2,1588319618908983296,Thu Nov 03 23:58:01 +0000 2022,A new @RemingtonArms Model 700 announcement sl...,Retweet,0,0,0,0,https://twitter.com/twitter/status/15883196189...
3,1588319519751041024,Thu Nov 03 23:57:37 +0000 2022,#Miami #Trump supporter convicted of shooting ...,Original Tweet,0,0,2,3,https://twitter.com/twitter/status/15883195197...
4,1588319508204490752,Thu Nov 03 23:57:34 +0000 2022,"Just before the midterm election, the White Ho...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15883195082...
...,...,...,...,...,...,...,...,...,...
95,1588299649437560833,Thu Nov 03 22:38:40 +0000 2022,"I don't know who needs to hear this, BUT..ther...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15882996494...
96,1588299190378979328,Thu Nov 03 22:36:50 +0000 2022,15-month-old boy’s death now ruled a homicide ...,Original Tweet,0,0,0,0,https://twitter.com/twitter/status/15882991903...
97,1588299067821621249,Thu Nov 03 22:36:21 +0000 2022,This doesn’t have to be our reality.\nVote aga...,Quote Tweet,0,0,6,3,https://twitter.com/twitter/status/15882990678...
98,1588298341602848771,Thu Nov 03 22:33:28 +0000 2022,@biggs1_terry @DrEliDavid Kind of a bold endor...,Reply Tweet,1,0,1,0,https://twitter.com/twitter/status/15882983416...


In [239]:
tweet_df.loc[33]['hyperlink']

'https://twitter.com/twitter/status/1588312826862452737'

In [240]:
tweet_df.loc[33]['full_text']

'"After several days of deliberation, a civil jury has found former Santa Clara County Sheriff Laurie Smith guilty of six corruption and willful misconduct charges, ending the sheriff’s nearly 50-year law enforcement career." https://t.co/Tj5A4CmNZd'

In [226]:
tweet_df2.loc[lambda df: df['tweet_type'] == 'Quote Tweet']

,tweet_id,date,full_text,tweet_type,reply_count,quote_count,likes_count,retweet_counts,hyperlink
33,1588312826862452737,Thu Nov 03 23:31:02 +0000 2022,"""After several days of deliberation, a civil j...",Quote Tweet,0,0,1,0,https://twitter.com/twitter/status/15883128268...
58,1588306757767467008,Thu Nov 03 23:06:55 +0000 2022,The crowd at a Trump rally in Iowa boos Paul P...,Quote Tweet,0,0,1,0,https://twitter.com/twitter/status/15883067577...
59,1588305989970718720,Thu Nov 03 23:03:51 +0000 2022,Method Man isn't trying to hide his frustratio...,Quote Tweet,1,0,0,0,https://twitter.com/twitter/status/15883059899...
97,1588299067821621249,Thu Nov 03 22:36:21 +0000 2022,This doesn’t have to be our reality.\nVote aga...,Quote Tweet,0,0,6,3,https://twitter.com/twitter/status/15882990678...


In [193]:
len(rawTweets)

100

In [191]:
tweet_df

,tweet_id,date,full_text,tweet_type,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1588320111932252160,Thu Nov 03 23:59:58 +0000 2022,"Just before the midterm election, the White Ho...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15883201119...
1,1588319976142041088,Thu Nov 03 23:59:26 +0000 2022,"Just before the midterm election, the White Ho...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15883199761...
2,1588319618908983296,Thu Nov 03 23:58:01 +0000 2022,A new @RemingtonArms Model 700 announcement sl...,Retweet,0,0,0,0,https://twitter.com/twitter/status/15883196189...
3,1588319519751041024,Thu Nov 03 23:57:37 +0000 2022,#Miami #Trump supporter convicted of shooting ...,Original Tweet,0,0,2,3,https://twitter.com/twitter/status/15883195197...
4,1588319508204490752,Thu Nov 03 23:57:34 +0000 2022,"Just before the midterm election, the White Ho...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15883195082...
...,...,...,...,...,...,...,...,...,...
95,1588299649437560833,Thu Nov 03 22:38:40 +0000 2022,"I don't know who needs to hear this, BUT..ther...",Retweet,0,0,0,0,https://twitter.com/twitter/status/15882996494...
96,1588299190378979328,Thu Nov 03 22:36:50 +0000 2022,15-month-old boy’s death now ruled a homicide ...,Original Tweet,0,0,0,0,https://twitter.com/twitter/status/15882991903...
97,1588299067821621249,Thu Nov 03 22:36:21 +0000 2022,This doesn’t have to be our reality.\nVote aga...,Quote Tweet,0,0,6,3,https://twitter.com/twitter/status/15882990678...
98,1588298341602848771,Thu Nov 03 22:33:28 +0000 2022,@biggs1_terry @DrEliDavid Kind of a bold endor...,Reply Tweet,1,0,1,0,https://twitter.com/twitter/status/15882983416...


In [130]:
len(pages[0])

58

In [131]:
len(pages)

1

In [132]:
print(json.dumps(pages[0][0]._json, indent=2))

{
  "created_at": "Tue Nov 08 16:47:57 +0000 2022",
  "id": 1590023329410613250,
  "id_str": "1590023329410613250",
  "text": "Just doing my part in #Texas \ud83d\ude4f\ud83c\udffc I was happy to cast my ballot for @betoorourke and fellow #democrats #downballot\u2026 https://t.co/I1qtXMgvQB",
  "source": "<a href=\"http://instagram.com\" rel=\"nofollow\">Instagram</a>",
  "truncated": true,
  "in_reply_to_status_id": null,
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id": null,
  "in_reply_to_user_id_str": null,
  "in_reply_to_screen_name": null,
  "user": {
    "id": 68327766,
    "id_str": "68327766",
    "name": "Elizabeth De Razzo",
    "screen_name": "ederazzo",
    "location": "San Antonio, TX",
    "url": "http://www.imdb.me/elizabethderazzo",
    "description": "Actress, Singer, Writer, Kelly Clarkson fan & Profesh Fangirl! Known for Eastbound & Down, IdiotSitter, Greasy Strangler, & Vida. She/Her.",
    "translator_type": "none",
    "protected": false,
    "verifi

In [89]:
print(json.dumps(response[4]._json, indent=2))

{
  "created_at": "Sun Nov 06 22:43:32 +0000 2022",
  "id": 1589388039750848512,
  "id_str": "1589388039750848512",
  "text": "My mother-in-law voted for @BetoORourke. First time she can remember voting for a democrat for gov. Said\u2026 https://t.co/RYzUVbdmQd",
  "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
  "truncated": true,
  "in_reply_to_status_id": null,
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id": null,
  "in_reply_to_user_id_str": null,
  "in_reply_to_screen_name": null,
  "user": {
    "id": 17909423,
    "id_str": "17909423",
    "name": "Rob Shearer",
    "screen_name": "robshearer",
    "location": "Dallas, TX",
    "url": "https://www.linkedin.com/in/rshearer/",
    "description": "Chief of Comms @unitedwaydallas. Formerly @commit2dallas. Married to @houseomacgregor. Citizen of Oak Cliff. Public school parent. Tweets are my own.",
    "translator_type": "none",
    "protected": false,
    "verified":

In [62]:
print(tweets_df['id'][0])

1589774640704286720


In [41]:
tweets_df = pd.json_normalize(response[0]._json)
tweets_df.T.head(50)

,0
created_at,Tue Nov 08 00:19:45 +0000 2022
id,1589774640704286720
id_str,1589774640704286720
text,New York! #GOTV - Election Day is tmrw - #Vote...
display_text_range,"[0, 112]"
source,"<a href=""http://twitter.com/download/iphone"" r..."
truncated,False
in_reply_to_status_id,None
in_reply_to_status_id_str,None
in_reply_to_user_id,None


In [42]:
tweets_df.T.tail(36)

,0
user.profile_background_image_url,http://abs.twimg.com/images/themes/theme20/bg.png
user.profile_background_image_url_https,https://abs.twimg.com/images/themes/theme20/bg...
user.profile_background_tile,False
user.profile_link_color,BF1238
user.profile_sidebar_border_color,FFFFFF
user.profile_sidebar_fill_color,EFEFEF
user.profile_text_color,333333
user.profile_use_background_image,True
user.profile_image_url,http://pbs.twimg.com/profile_images/1548431537...
user.profile_image_url_https,https://pbs.twimg.com/profile_images/154843153...


# API V2 CODE BELOW

In [294]:
# Twitter Authentication
client = tweepy.Client(bearer_token=twit_token)

In [ ]:
query = '(#guncontrol OR #gunrights OR #gunsense OR #2a OR #gunviolence) lang:en -is:retweet'
counts = client.get_recent_tweets_count(query=query, granularity='day')

for i in counts.data:
    print(i["tweet_count"])

In [153]:
# Fetch recent public tweets from twitter on a keyword (#Uvalde)
query = '#Uvalde -is:retweet lang:en'
paginator = tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            #expansions=geo.place_id,
            # tweets per page
            max_results=90,
            # number of pages to retrieve
            limit=1
        )
        # Get tweets that contain the hashtag #Uvalde
        # place_country:US to only select tweets made from United States (unavailable in current access level)
        # -is:retweet to exclude retweets
        # lang:en only tweets in the english language

# Append tweets to list
tweet_list = []

for tweet in paginator.flatten():
    tweet_list.append(tweet)

In [154]:
len(tweet_list)

90

In [17]:
# Creating new DataFrame for #Uvalde tweets
htUvalde_tweets_df = pd.DataFrame(tweet_list)
htUvalde_tweets_df = pd.DataFrame(htUvalde_tweets_df['text'])
htUvalde_tweets_df.head(5)

,text
0,My daughters and I voted today for you....did ...
1,SHAME on YOU @GregAbbott_TX ‼️How do you sleep...
2,Greg Abbott can’t and won’t keep your kids saf...
3,Caitlyne shares a name with my youngest\n\nIf ...
4,Like millions of @MomsAGAbbott &amp; @MomsDema...


In [18]:
# Fetch recent public tweets from twitter on a keyword (#guncontrol)
query = '#guncontrol -is:retweet lang:en'
paginator = tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            #expansions=geo.place_id,
            # tweets per page
            max_results=10,
            # number of pages to retrieve
            limit=5
        )
        # Get tweets that contain the hashtag #guncontrol
        # place_country:US to only select tweets made from United States (unavailable in current access level)
        # -is:retweet to exclude retweets
        # lang:en only tweets in the english language

# Append tweets to list
tweet_list = []

for tweet in paginator.flatten():
    tweet_list.append(tweet)
    print(tweet)

Forgetting #Uvalde FAST as SHOCKING Texas Rule on (LACK) of #GunControl Dominates: #Texas Goes Permitless on Guns &amp; Police Face Armed Public #EndGunViolence #GunReformNOW #GunSense #StopGunViolence #GunControlNOW #IARTG #Progressives #BTRTG #GunViolence  https://t.co/OiIcytSWB9
@JasonKander @JesseKellyDC The irony of the profiteering Right's use of the "death by thousand cuts" analogy whenever they lose ground in #GunControl while continually excusing any bigotry, misogyny, antisemitism, etc. as an "anomoly." It's a dangerous path a strong America would be wise to avoid
Hi @GovRonDeSantis Burgundy?! 
::Fix The Teleprompter::

Hi Mr DeSantis! Would you have any other documentation on your political positions. I am making my decisions for #ElectionDay 
I would like to hear more about #GunControl #environmental &amp; how you can help us? https://t.co/8C8DxNlgGB
Modern day #journalism be like "Most Americans want more #guncontrol." Okay, what percent of the 332 million #Americans did y

In [20]:
# Creating new DataFrame for #guncontrol tweets
htguncontrol_tweets_df = pd.DataFrame(tweet_list)
htguncontrol_tweets_df = pd.DataFrame(htguncontrol_tweets_df['text'])
htguncontrol_tweets_df.head(5)

,text
0,Forgetting #Uvalde FAST as SHOCKING Texas Rule...
1,@JasonKander @JesseKellyDC The irony of the pr...
2,Hi @GovRonDeSantis Burgundy?! \n::Fix The Tele...
3,"Modern day #journalism be like ""Most Americans..."
4,@MarshaBlackburn Gun deaths per capita are HIG...


In [26]:
# Concatenating #Uvalde and #guncontrol tweets DataFrames into a single DataFrame
combined_tweets_df = pd.concat([htUvalde_tweets_df, htguncontrol_tweets_df])
# Reset DataFrame index after concatenating
combined_tweets_df.reset_index(drop=True, inplace=True)

# Check DataFrame
combined_tweets_df.head()

,text
0,My daughters and I voted today for you....did ...
1,SHAME on YOU @GregAbbott_TX ‼️How do you sleep...
2,Greg Abbott can’t and won’t keep your kids saf...
3,Caitlyne shares a name with my youngest\n\nIf ...
4,Like millions of @MomsAGAbbott &amp; @MomsDema...


In [27]:
# Confirm concatenation and reindexing by checking middle of DataFrame
combined_tweets_df.loc[45:55]

,text
45,I voted in the best interest of these babies a...
46,@AbbottCampaign @GregAbbott_TX I Voted Early f...
47,@GregAbbott_TX @AngieChenButton @TanParkerTX I...
48,@TomCottonAR Crime. Crime. Crime. #highlandpar...
49,A painful reminder of why we need to Vote agai...
50,Forgetting #Uvalde FAST as SHOCKING Texas Rule...
51,@JasonKander @JesseKellyDC The irony of the pr...
52,Hi @GovRonDeSantis Burgundy?! \n::Fix The Tele...
53,"Modern day #journalism be like ""Most Americans..."
54,@MarshaBlackburn Gun deaths per capita are HIG...


In [33]:
# Check for duplicate tweets (recent tweets with both hashtags)
duplicates = combined_tweets_df.duplicated()
duplicates.value_counts()

False    100
dtype: int64

In [34]:
combined_tweets_df.to_csv('#Uvalde_#guncontrol_raw_tweets.csv')